In [1]:
import numpy as np
import pandas as pd
from scipy.io import wavfile
import joblib
import seaborn as sns
%matplotlib tk
#%matplotlib qt
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from pylab import specgram
from scipy.signal import argrelmax
from scipy.stats import linregress, pearsonr
import os

#%pylab inline
import cv2


Autosaving every 180 seconds


In [2]:

def read_filter_data(fname, call_onset, call_offset, filter_data, plotting):

    # load data
    print ("Loading: ", fname)
    data = np.load(fname)
    mic0_audio = data[0]
    mic1_audio = data[1]
    mic2_audio = data[2]
    mic3_audio = data[3]
    
    # clip data 
    #print ("Clipping :", call_onset, call_offset)
    fs = 250000
    call_mic0 = mic0_audio[int(call_onset*fs):int(call_offset*fs)]
    call_mic1 = mic1_audio[int(call_onset*fs):int(call_offset*fs)]
    call_mic2 = mic2_audio[int(call_onset*fs):int(call_offset*fs)]
    call_mic3 = mic3_audio[int(call_onset*fs):int(call_offset*fs)]  


    nfft = 1024
    if plotting:
        fig=plt.figure()
        ax1=plt.subplot(4,2,1)
        pxx, freqs, bins, im = specgram(call_mic0, NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-100,-80))
        plt.ylim(0,40000)
        plt.ylabel('mic 0')

        ax1=plt.subplot(4,2,3)
        pxx, freqs, bins, im = specgram(call_mic1, NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-100,-80))
        plt.ylim(0,40000)
        plt.ylabel('mic 1')

        ax1=plt.subplot(4,2,5)
        pxx, freqs, bins, im = specgram(call_mic2, NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-100,-80))
        plt.ylim(0,40000)
        plt.ylabel('mic 2')

        ax1=plt.subplot(4,2,7)
        pxx, freqs, bins, im = specgram(call_mic3, NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-100,-80))
        plt.ylim(0,40000)
        plt.ylabel('mic 3')

    # filter data
    start_fs = 6500
    end_fs = 7500
    mic0_audio_bp = butter_bandpass_filter(call_mic0, start_fs, end_fs, fs)  # YOU CAN AUTODETECT THIS
    mic1_audio_bp = butter_bandpass_filter(call_mic1, start_fs, end_fs, fs)
    mic2_audio_bp = butter_bandpass_filter(call_mic2, start_fs, end_fs, fs)
    mic3_audio_bp = butter_bandpass_filter(call_mic3, start_fs, end_fs, fs)

    if plotting:
        ax2=plt.subplot(4,2,2)
        pxx, freqs, bins, im = specgram(mic0_audio_bp, NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-100,-80))
        plt.ylim(0,40000)

        ax2=plt.subplot(4,2,4)
        pxx, freqs, bins, im = specgram(mic1_audio_bp, NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-100,-80))
        plt.ylim(0,40000)

        ax2=plt.subplot(4,2,6)
        pxx, freqs, bins, im = specgram(mic2_audio_bp, NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-100,-80))
        plt.ylim(0,40000)

        ax2=plt.subplot(4,2,8)
        pxx, freqs, bins, im = specgram(mic3_audio_bp, NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-100,-80))
        plt.ylim(0,40000)

        plt.suptitle(os.path.split(fname)[1])
    
    if filter_data:
        return (mic0_audio_bp, mic1_audio_bp, mic2_audio_bp, mic3_audio_bp)
    else:
        return (call_mic0, call_mic1, call_mic2, call_mic3)
    
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y



In [27]:
# Load video file
fname = "/media/cat/4TBSSD/dan/july_19/2020_07_24_15_11_26_425064/video/2020_07_24_15_11_26_425064.avi"

fname_light = fname[:-4]+"_light.npy"
fname_vid = fname[:-4]+".npy"
if os.path.exists(fname_light)==False:
    cap = cv2.VideoCapture(fname)

    # make a downsampled version of the video; Even more should be ok
    vid = np.zeros((30000,1024//4,1280//4,3),'uint8')

    ctr=0
    while(cap.isOpened()):
        if ctr%1000==0:
            print (ctr)
        ret, frame = cap.read()

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        vid[ctr]=frame[::4,::4]
        ctr+=1

    cap.release()
    cv2.destroyAllWindows()
    print ("Video file: ", vid.shape)
    # Find the pixel with LED light and track it's max over all 3 color channels over time
    light = vid[:,147, 306].max(1)
    print(light.shape)
    np.save(fname_light, light)
    np.save(fname_vid, vid)
else:
    light = np.load(fname_light)
    vid = np.load(fname_vid)
print ("LED Light: ", light.shape)
print ("Vid: ", vid.shape)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


TypeError: 'NoneType' object is not subscriptable

In [14]:
# Load audio file
mic4 = np.load('/media/cat/4TBSSD/dan/july_19/2020_07_24_15_11_26_425064/audio/2020_07_24_15_11_26_425064_audio_mic_4.npy')
print ("audio file: ", mic4.shape)

audio file:  (150000000,)


In [15]:
# Visualize data against each other OPTIONAL
if False:
    t_led = np.arange(light.shape[0])/25.
    temp = light-light.mean(0)
    plt.plot(t_led, temp/50., c='red')

    # plot mic ttl pulse
    mic4_subsampled = mic4[::10]
    t_mic = np.arange(mic4_subsampled.shape[0])/12500.
    temp = mic4_subsampled-mic4_subsampled.mean(0)
    plt.plot(t_mic, temp+2.8,c='blue')

    # formating
    plt.title("LED: red;   TTL Pulse: Blue", fontsize=20)
    plt.xlim(0,1200)
    plt.yticks([])
    plt.xlabel("Time (sec)",fontsize=20)
    plt.show()

In [16]:
# TODO: PLOT THE DISTRIBUTION OF DISTANCES BETWEEN LED LIGHT AND TTL AUDIO
# THIS WILL INDICATE WHETHER STRETECHING OF DATA Is required

In [17]:
mic0 = np.load('/media/cat/4TBSSD/dan/july_19/2020_07_24_15_11_26_425064/audio/2020_07_24_15_11_26_425064_audio_mic_0.npy')
print (mic0.shape)

(150000000,)


In [99]:
nfft = 1024
fs_audio = 125000
fs_camera = 25

start = int((400.98+8.7+1.4+9.08+2.47+0.35)*fs)
end = int(start+.1*fs)

print (start/fs, end/fs, 'sec')

ax=plt.subplot(1,2,1)
pxx, freqs, bins, im = specgram(mic0[start:end], NFFT=nfft, Fs=fs, noverlap=nfft/2,
                                       clim=(-120,-60))
plt.title("Start: "+str(start/fs_audio)+ " end: "+str(end/fs_audio)+" sec")
# 
ax=plt.subplot(1,2,2)
frame_id_camera = int(start/fs*fs_camera)
print (frame_id_camera)
plt.imshow(vid[frame_id_camera])
plt.title("Frame: "+str(frame_id_camera))
plt.show()


422.98 423.08 sec
10574


In [100]:
fname = '/media/cat/4TBSSD/dan/july_19/2020_07_24_15_11_26_425064/snippets/2020_07_24_15_11_26_425064_snippet_'

snippet=9
np.savez(fname+str(snippet), 
        usv = mic0[start:end],
        frame = vid[frame_id_camera],
        time_start_sec = start,
        time_end_sec = end)

In [16]:
# COMPUTE SHIFT CORRELATION AND FIT POLYNOMIAL

mics = read_filter_data(fname, call_onsets[ctr],call_offsets[ctr], filter_data, plotting)
    

    

[520 780 520 780]
Loading:  E:\july10_pos8A\audio\2020-07-10_20_05_40.673203_audio.npy


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:211: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


len r2 array:  4
Loading:  E:\july10_pos8B\audio\2020-07-10_20_06_22.532042_audio.npy
len r2 array:  4
Loading:  E:\july10_pos8C\audio\2020-07-10_20_07_19.409833_audio.npy
len r2 array:  4
Loading:  E:\july10_pos8D\audio\2020-07-10_20_08_02.677607_audio.npy
len r2 array:  4
Loading:  E:\july10_pos8E\audio\2020-07-10_20_09_07.645162_audio.npy
len r2 array:  4
Loading:  E:\july10_pos1\audio\2020-07-10_20_01_52.787211_audio.npy
len r2 array:  4
Loading:  E:\july10_pos5\audio\2020-07-10_20_03_05.256554_audio.npy
len r2 array:  4
Loading:  E:\july10_pos11\audio\2020-07-10_20_04_20.196753_audio.npy
len r2 array:  4
Loading:  E:\july10_pos15\audio\2020-07-10_20_03_42.148538_audio.npy
len r2 array:  4
DONE


# bandpass filter

In [14]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

start_fs = 6500
end_fs = 7500
mic0_audio_bp = butter_bandpass_filter(mic0_audio, start_fs, end_fs, fs)  # YOU CAN AUTODETECT THIS
mic1_audio_bp = butter_bandpass_filter(mic1_audio, start_fs, end_fs, fs)
mic2_audio_bp = butter_bandpass_filter(mic2_audio, start_fs, end_fs, fs)
mic3_audio_bp = butter_bandpass_filter(mic3_audio, start_fs, end_fs, fs)

In [10]:
plt.figure(figsize=(20,3))
ax=plt.subplot(1,1,1)
plt.plot(call_mic0_bp, 'k', markersize=1)
#plt.title('mic0')

#plt.figure(figsize=(20,3))
#ax=plt.subplot(2,1,2)
plt.plot(call_mic1_bp, 'r', markersize=1)
plt.title('mic1')

Text(0.5, 1.0, 'mic1')